In [65]:
import numpy as np
import csv
import os
import math
import matplotlib.pyplot as plt

os.chdir('/Users/sreelakshmirajendrakumar/Downloads')


# reading the csv file
x=[]
for i in range(0,100):
    x.append([])
with open('OP_train1.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        try:
            for i in range(1,101):
                x[i-1].append(float(row[i]))
        except:
            row==0
Y=[]            
with open('OP_train1.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        try:
            Y.append(float(row[101]))
        except:
            row==0
            
X=[]           
for i in range(0,51):
    if i!=0:
        X.append([])
        for j in range(0,100):
            X[i-1].append(x[j][i])
            

def multivariate(l):
    d=100
    N=50            
    #restructuring the X_test file
    
    W_T=[]
    Z_T=[]
    for i in range(0,d):
        W_T.append([])
        Z_T.append([])
        index_0=[]
        index_1=[]
        for j in range(0,N):
            if Y[j]==0:
                W_T[i].append(X[j][i])
                index_0.append(j)
            else:
                Z_T[i].append(X[j][i])
                index_1.append(j)

    w=np.transpose(W_T)
    z=np.transpose(Z_T)

    # finding u_(1,0) and U_(1,1)
    u_10=[]
    for i in range(0,d):
        sum=0
        for j in range(0,len(index_0)):
            sum+=w[j][i]
        if len(index_0)!=0:
            sum=sum/len(index_0)
        u_10.append(sum)
    
    
    u_11=[]
    for i in range(0,d):
        sum=0
        for j in range(0,len(index_1)):
            sum+=z[j][i]
        if len(index_1)!=0:
            sum=sum/len(index_1)
        u_11.append(sum)
    
        
    # finding u_1 and u_2
    u_1=[]
    u_2=[]
    for i in range(0,d):
        sum=0
        sum_2=0
        for j in range(0,N):
            sum+=X[j][i]
            sum_2+=((X[j][i])**2)
        u_1.append(sum/N)
        u_2.append(sum_2/N)

    # apply the equation --> 1. checking the value of lambda and element-wise opeation on the matrices

    N_0=len(index_0)
    N_1=len(index_1)

    condition=[]
    for i in range(0,d):
        c=((N_0*N_1/(N**2))*(abs(u_11[i]-u_10[i])/np.sqrt(u_2[i]-(u_1[i]**2))))
        condition.append(c)
    
    m_0=[]
    m_1=[]
    variance=[]
    sigma=[]
    for i in range(0,d):
        if l>=condition[i]:
            m_0.append(u_1[i])
            m_1.append(u_1[i])
            temp=(u_2[i]-(u_1[i]**2))
            variance.append(temp)
            sigma.append(np.sqrt(temp))
        
        elif l<=condition[i]:
            s=u_2[i]-(u_1[i]**2)-((N_0*N_1/N**2)*((u_11[i]-u_10[i])**2))
            sig=abs(u_11[i]-u_10[i])*l+np.sqrt(((u_11[i]-u_10[i])**2)*(l**2)+4*s)
            sigma.append(np.sqrt(sig))
            if u_11[i]>=u_10[i]:
                temp1=u_10[i]+(((N_1/N)*(u_11[i]-u_10[i]))*sig*l)
                temp2=u_11[i]-(((N_0/N)*(u_11[i]-u_10[i]))*sig*l)
            else:
                temp1=u_10[i]+(((N_1/N)*(u_10[i]-u_11[i]))*sig*l)
                temp2=u_11[i]-(((N_0/N)*(u_10[i]-u_11[i]))*sig*l)
            m_0.append(temp1)
            m_1.append(temp2)
        
    g=0
    for i in range(0,100):
        if m_0[i]==m_1[i]:
            g+=1
    print('The required number of coordinates is '+str(g)+ ' when lambda = ' +str(l))
    return m_0,m_1,sigma   
        

In [66]:
#when lambda=.1
m_0_1,m_1_1,sigma_1=multivariate(.1)
#when lambda=.15
m_0_2,m_1_2,sigma_2=multivariate(.15)



The required number of coordinates is 86 when lambda = 0.1
The required number of coordinates is 97 when lambda = 0.15


In [67]:
# Question
x_test=[]
for i in range(0,100):
    x_test.append([])
with open('OP_test1.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        try:
            for i in range(1,101):
                x_test[i-1].append(float(row[i]))
        except:
            row==0
Y_test=[]            
with open('OP_test1.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        try:
            Y_test.append(float(row[101]))
        except:
            row==0
            
X_test=[]           
for i in range(0,51):
    if i!=0:
        X_test.append([])
        for j in range(0,100):
            X_test[i-1].append(x[j][i])

Y_new=np.zeros(N)


In [68]:
#finding error when lambda=.1
for i in range(0,N):
    comp1=0
    comp2=0
    for j in range(0,d):
        comp1+=(((X[i][j]-m_0_1[j])**2)/(sigma_1[j]**2))
        comp2+=(((X[i][j]-m_1_1[j])**2)/(sigma_1[j]**2))
    if comp1<comp2:
        Y_new[i]=0
    else:
        Y_new[i]=1
        
error_rate=0
for i in range(0,N):
    if Y_new[i]!=Y_test[i]:
        error_rate+=1
error=error_rate/50

print('The error when lambda=.1 is '+str(error))

The error when lambda=.1 is 0.48


In [69]:
#finding error when lambda=.15
for i in range(0,N):
    comp1=0
    comp2=0
    for j in range(0,d):
        comp1+=(((X[i][j]-m_0_2[j])**2)/(sigma_2[j]**2))
        comp2+=(((X[i][j]-m_1_2[j])**2)/(sigma_2[j]**2))
    if comp1<comp2:
        Y_new[i]=0
    else:
        Y_new[i]=1
        
error_rate=0
for i in range(0,N):
    if Y_new[i]!=Y_test[i]:
        error_rate+=1
error=error_rate/50

print('The error when lambda=.15 is '+str(error))

The error when lambda=.15 is 0.48
